<a href="https://colab.research.google.com/github/luke-who/Digital-Twin-4th-Year/blob/main/test/digital_twin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing OS

In [ ]:
!apt install screenfetch &
!screenfetch

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

## creating a tensor of 3 rows and 2 columns consisting of ones
x = torch.ones(3,2)

## creating a tensor of 3 rows and 2 columns consisting of zeros
x = torch.zeros(3,2)

# To increase the reproducibility, we often set the random seed to a specific value first.
torch.manual_seed(2)
x = torch.rand(3,2) #generating tensor randomly
x1 = torch.randn(3,2) #generating tensor randomly from normal distribution

# print(x[:,-1])
# print(x[0,:])
# print(x[1,1])

y = x.view(2,3) #reshaping x to 2 rows and 3 columns
y = x.view(6,-1) #y shape will be 6x1 as -1 is inferred from the first dimension

#adding/substracting two tensors
z = x + x1
z = torch.add(x,x1)

z = x - x1
z = torch.sub(x,x1)

# Inplace operation
x.add_(x1) #tensor x added with x1 and result will be stored in x

# convert pytorch to numpy
x = torch.linspace(0,1,steps=5)
x_np = x.numpy()
# print(type(x),type(x_np))

# convert numpy to tensor
a = np.random.randn(5)
a_pt = torch.from_numpy(a)
# print(type(a),type(a_pt))

print(torch.cuda.device_count()) #print the number of GPUs
print(torch.cuda.get_device_name(0)) #et the name of the GPU Card connected to the machine

#Assign cuda GPU located at location '0' to a variable
cuda0 = torch.device('cuda:0')
#Performing the addition on GPU
a = torch.ones(3,2,device=cuda0)
b = torch.ones(3,2,device=cuda0)
c = torch.add(a,b)
#moving the result to cpu
c = c.cpu()

# Automatic Differentiation
x = torch.ones([3,2],requires_grad = True)
y = x + 5
z = y*y + 1
t = torch.sum(z) #adding all the values in z
t.backward()
print(x.grad)

1
Tesla T4
tensor([[12., 12.],
        [12., 12.],
        [12., 12.]])


In [ ]:
# Import libraries
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Create test and training sets
train = datasets.MNIST('', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

test = datasets.MNIST('', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))


# This section will shuffle our input/training data so that we have a randomized shuffle of our data and do not risk feeding data with a pattern. 
# Anorther objective here is to send the data in batches. This is a good step to practice in order to make sure the neural network does not overfit our data. 
# NN’s are too prone to overfitting just because of the exorbitant amount of data that is required. For each batch size, the neural network will run a back propagation for new updated weights to try and decrease loss each time.
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)


# Initialize our neural net
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net = Net()

print(net)

### Output:
### Net(
###  (fc1): Linear(in_features=784, out_features=64, bias=True)
###  (fc2): Linear(in_features=64, out_features=64, bias=True)
###  (fc3): Linear(in_features=64, out_features=64, bias=True)
###  (fc4): Linear(in_features=64, out_features=10, bias=True)
###)


# Calculate our loss 
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

X,y = testset.dataset[0]
print(X.size(),y)

for epoch in range(5): # we use 5 epochs
    for data in trainset:  # `data` is a batch of data
        X, y = data  # X is the batch of features, y is the batch of targets.

        net.zero_grad()  # sets gradients to 0 before calculating loss.

        output = net(X.view(-1,784))  # pass in the reshaped batch (recall they are 28x28 atm, -1 is needed to show that output can be n-dimensions. This is PyTorch exclusive syntax)

        loss = F.nll_loss(output, y)  # calc and grab the loss value

        loss.backward()  # apply this loss backwards thru the network's parameters

        optimizer.step()  # attempt to optimize weights to account for loss/gradients
    print(loss)  

### Output:
### tensor(0.6039, grad_fn=)
### tensor(0.1082, grad_fn=)
### tensor(0.0194, grad_fn=)
### tensor(0.4282, grad_fn=)
### tensor(0.0063, grad_fn=)


# Get the Accuracy
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1,784))
        #print(output)
        for idx, i in enumerate(output):
            #print(torch.argmax(i), y[idx])
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print("Accuracy: ", round(correct/total, 3))

### Output: 
### Accuracy:  0.915

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)
tensor(0.2599, grad_fn=<NllLossBackward0>)
tensor(0.3964, grad_fn=<NllLossBackward0>)
tensor(0.1890, grad_fn=<NllLossBackward0>)
tensor(0.1908, grad_fn=<NllLossBackward0>)
tensor(0.1782, grad_fn=<NllLossBackward0>)
Accuracy:  0.917


# Test VAE